In [7]:
# -*- coding: utf-8 -*-
#! /usr/bin/env python3
# STEP1: Get audio URL from XiMaLaYa, download with BitComet
# STEP2: Run dos batch rename audio files 
# STEP3: Run loadMP3

# %%
import json
import os
import re
import urllib.request
import socket
import shutil
import glob
from bs4 import BeautifulSoup
from math import log10
import codecs
import time
import random

LocalDir = "mp3"
#beginUrlSets = ['/renwen/220566/'] # 西游记 原文朗读
#beginUrlSets = ["/lishi/13396678/"] #易中天品三国
#beginUrlSets = ["/renwen/220565/"] # 红楼梦 原文朗读
#beginUrlSets = ["/renwen/268522/"] # 三国演义 原文朗读
#beginUrlSets = ["/xiangsheng/16313345/"] #三国演义
#beginUrlSets = ['/renwen/2798291/'] # 水浒传 原文朗读
# beginUrlSets = ['/youshengshu/3474810/'] #简爱
beginUrlSets = ['/yinyue/412359/'] # music

# False - M4A url to url-download.txt, rename DOS batch rename.bat.txt
# True - download M4A right now
downloadM4A = True # False
delayInSecond = 10
albumId = ''
urlSetFinishedPages = set()
urlSetUnfinishedPages = set()
urlSetErrorPages = set()
urlSetErrorAudio = set()

def main():
    global albumId
    baseUrl = "http://www.ximalaya.com"        
    for url in beginUrlSets:
        urlSetUnfinishedPages.add(url)
    
    socket.setdefaulttimeout(100)    
    while( len(urlSetUnfinishedPages) > 0 ):
        print( "unfinished pages:" + str(urlSetUnfinishedPages) )
        unfinishedUrl = urlSetUnfinishedPages.pop()
        fullUrl = unfinishedUrl
        if( unfinishedUrl.find( "http://") != 0 ):
            fullUrl =  baseUrl + unfinishedUrl #urljoin.url_path_join( baseUrl, unfinishedUrl )
        bsObj = bsObjForm( fullUrl )
        if bsObj:
            urlSetFinishedPages.add( unfinishedUrl )
        else:
            urlSetErrorPages.add(unfinishedUrl)
            continue
        getAlbumPages( bsObj, urlSetFinishedPages, urlSetUnfinishedPages )
        albumJson = getAlbumPageInfo( bsObj )
        if( albumJson == None ):
            urlSetErrorPages.add(unfinishedUrl)
            continue
    
        albumInfoJson = albumJson["AlbumDetailPage"]["albumInfo"]
        albumId = albumInfoJson["albumId"]
        albumCoverImgUrl = albumInfoJson["mainInfo"]["cover"]
        albumTitle = albumInfoJson["mainInfo"]["albumTitle"]
        albumPlayCount = albumInfoJson["mainInfo"]["playCount"]
        albumIsFinished = albumInfoJson["mainInfo"]["isFinished"]
        print( "album id:" + str(albumId) + " Cover:" + albumCoverImgUrl + " Title:" + albumTitle + " PlayCount:" + str(albumPlayCount) + " IsFinished:" + str(albumIsFinished) )
    
        allTrackNum = int(albumJson["AlbumDetailTrackList"]["tracksInfo"]["trackTotalCount"])
        print( "All track num:" + str( allTrackNum ) )
        tracksInfoArray = albumJson["AlbumDetailTrackList"]["tracksInfo"]["tracks"]
    
        indexWidth = int( 0.5 + log10(allTrackNum) ) + 1
        indexFormatStr = "%0" + str(indexWidth) + "d"
    
        for tracksInfo in tracksInfoArray:
            trackId = str(tracksInfo["trackId"])
            trackIndex = indexFormatStr%int(tracksInfo["index"])
            trackName = tracksInfo["title"]
            isPaid = str(tracksInfo["isPaid"])
            print( "track index:" + trackIndex + " id:" + trackId + " title:" + trackName + " isPaid:" + isPaid )
            if( isTrackExist( albumTitle, trackName, trackIndex, "") ):
                continue
            else:
                getTrackSoundUrl( trackId, trackIndex )
    
    print( "Finished urls:" + str(urlSetFinishedPages) )
    print( "Error urls:" + str( urlSetErrorPages ) )
    print( "Error Audio urls:" + str( urlSetErrorAudio ))


def bsObjForm(url):
    print("Request url:" + url)
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) ' 'Chrome/51.0.2704.63 Safari/537.36'}
    req = urllib.request.Request(url=url, headers=headers)
    html = urllib.request.urlopen(req).read().decode('utf-8','ignore')#, 'ignore'
    bsObj1 = BeautifulSoup(html, "html.parser")
    return bsObj1

def getTrackLocalPath( albumTitle, trackTitle, trackIndex, fileExt ):
    albumTitle= re.sub('[\/:*?"<>|]','_',albumTitle)
    trackTitle= re.sub('[\/:*?"<>|]','_',trackTitle)
    if fileExt == "":
        fileExt = ".m4a"
    article_path= os.path.join(os.path.join(LocalDir, albumTitle), trackIndex + "-" + trackTitle+ fileExt)
    return  article_path

def isTrackExist( albumTitle, trackTitle, trackIndex, fileExt ):
    articalPath = getTrackLocalPath( albumTitle, trackTitle, trackIndex, fileExt )
    if  os.path.exists(articalPath):
        print('%s 文件已经存在'%articalPath)
        return  True
    else:
        return False

def Schedule(a,b,c):
    '''''
    a:已经下载的数据块
    b:数据块的大小
    c:远程文件的大小
   '''
    per = 100.0 * a * b / c
    if per > 100 :
        per = 100
    #print ('%.2f%%' % per,'已下载的大小:',a*b,'文件大小:',c)
    #print '已经下载的数据块:',a#,'\n'
    #print '数据块的大小:',b#,'\n'
    #print '远程文件大小:',c,'\n'
    #print '已下载的大小:',a*b,'文件大小:',c

def createOutputTxt(play_path, article_path):
    fn_xmly = play_path.split('/')[-1]
    fn_local = os.path.basename(article_path)
    fn_urlsets = os.path.join(LocalDir, r'url-sets-download_' + str(albumId) + '.txt')
    fn_dos = os.path.join(LocalDir, r'rename_w4a_' + str(albumId) + '.bat.txt')
    
    with open(fn_urlsets, 'a') as fout:
        fout.write(play_path + '\n')
    with codecs.open(fn_dos, 'a', 'utf') as fout:
        fout.write('ren {} {}\n'.format(fn_xmly, fn_local.replace(' ','_')))
   
#获得专辑信息，分析音频文件名称，所属专辑，下载地址，并将文件保存在本地
def getM4a(url, trackIndex, downloadMode):
    print( "Get track file url:" + url + " index:" + trackIndex )
    bsObj = bsObjForm(url)
    soundInfo=bsObj.text
    try:
        jsonStr = json.loads(soundInfo)
    except:
        print( "load json fail! str:" + soundInfo + " url:" + url  )
        return
    album_title=jsonStr['album_title']
    play_path=jsonStr["play_path"]
    title=jsonStr["title"]

    fileExt = os.path.splitext(play_path)[1]
    if( fileExt == "" ):
        fileExt = ".m4a"

    #由于title里面可能包含不能作文件名称的字符，所以将这些字符去掉
    title= re.sub('[\/:*?"<>|]','_',title)
    album_title = re.sub('[\/:*?"<>|]','_',album_title)
    print(album_title,play_path,title)
    #将单曲写入文件
    #判断文件夹是否存在，如果不存在，则新建文件夹
    if os.path.exists(os.path.join(LocalDir, album_title))==False:
        os.makedirs(os.path.join(LocalDir, album_title))

    article_path= getTrackLocalPath( album_title, title, trackIndex, fileExt)
    article_path = article_path.replace(' ','_')
    
    print( "m4a url:" + play_path + "\tlocal path:" + article_path )
    
    if downloadMode:
        #判断文件是否存在，如果存在，则不进行下载
        if  os.path.exists(article_path):
            print('%s 文件已经存在'%title)
        else:
            time.sleep(random.randrange(delayInSecond-5, delayInSecond+5))
            try:
                urllib.request.urlretrieve(play_path,article_path,Schedule)
            except Exception as e:
                print('!! Error on getting:\n' +  play_path + '|' + article_path)
                urlSetErrorAudio.add(play_path + '|' + article_path)
                                
    else:
        createOutputTxt(play_path, article_path)
#        with open(r'c:\tmp\url.txt', 'a') as fout:
#            fout.write(play_path + "||" + article_path + '\n')



def getAlbumPages( soap, urlSetFinishedPages, urlSetPages ):
    pageArray = soap.select(".page-link")
    for pageObj in pageArray:
        pageUrl = pageObj.attrs['href']
        urlSetPages.add(pageUrl)
        urlSetPages -= urlSetFinishedPages

def getTrackSoundUrl( trackId, trackIndex ):
    trackInfoUrl = 'http://www.ximalaya.com/tracks/'+trackId+'.json' #"http://www.ximalaya.com/revision/play/tracks?trackIds=" + trackId;
    getM4a( trackInfoUrl, trackIndex, downloadM4A )

#分析所属专辑内各文件地址
def getAlbumPageInfo( bsObj ):
    scriptList = bsObj.select('html > body > script')
    for scriptObj in scriptList:
        if( scriptObj.text.find("window.__INITIAL_STATE__ = ") != -1 ):
            albumJsonStr = re.findall("{.*}}", scriptObj.text)[0]
            albumJson = json.loads( albumJsonStr )
            return albumJson
    return ''

if __name__ == "__main__":
    main()

# %%
def loadMP3():

    import os
    import glob
    import shutil
    
    source_dir = r'C:\tmp\MP3'    
#    source_dir = r'C:\Downloads\xmly\YangJiaJiang'
    target_dir = 'f:\\'
#    ext = r'*.m4a'
    ext = r'*.mp3'
    
    cnt = 0
    search_fullpath = os.path.join(source_dir, ext)
    files_list = glob.glob(search_fullpath)
    files_list.sort()
    for fn in files_list:
        shutil.copy(fn, target_dir)
        cnt += 1
        print(cnt, fn)
        
    print('Load {} files.'.format(str(cnt)))   

# %% Rename audio files
def renameAudioFiles():
    import os
    import codecs
    
    albumId = r'3474810'
    path = r'C:\Downloads\$$'
    bat = r'C:\Downloads\$$\rename_w4a_$$.bat.txt'    
    path = path.replace('$$', albumId)
    bat = bat.replace('$$', albumId)
    
    with codecs.open(bat, 'r', 'utf-8') as fin:
        lines = fin.readlines()
        
    failFileList = []
    
    for li in lines:
        fn = li.strip().split(' ') 
        print(li)     
        if len(fn)>1:
            print(fn)
            src = path + "\\" + fn[1]
            des = path + "\\" + fn[2]
            if os.path.exists(src):
                os.rename(src, des)
                print('{} >> {}'.format(src, des))
            else:
                failFileList.append(fn[1])
                print('!!!No file:' + fn[1])   
    print('==========DONE===========')            
    if len(failFileList)>0:
        print(failFileList)
    else:
        print('Succed!!')
        
    
    


unfinished pages:{'/yinyue/412359/'}
Request url:http://www.ximalaya.com/yinyue/412359/
album id:412359 Cover://imagev2.xmcdn.com/group14/M03/01/75/wKgDY1VKwufysbUpAAG856mg9OU752.jpg!op_type=5&upload_type=album&device_type=ios&name=mobile_large&strip=1&quality=7 Title:最新最火最好听流行歌曲 PlayCount:5290367 IsFinished:0
All track num:16
track index:16 id:8116259 title:我一个人记得就好 - 品冠 isPaid:False
Get track file url:http://www.ximalaya.com/tracks/8116259.json index:16
Request url:http://www.ximalaya.com/tracks/8116259.json
最新最火最好听流行歌曲 http://audio.xmcdn.com/group8/M08/56/F4/wKgDYVXAE_fBmchFABzephy0UJc158.m4a 我一个人记得就好 - 品冠
m4a url:http://audio.xmcdn.com/group8/M08/56/F4/wKgDYVXAE_fBmchFABzephy0UJc158.m4a	local path:mp3/最新最火最好听流行歌曲/16-我一个人记得就好_-_品冠.m4a
track index:15 id:8116248 title:全民共舞 - 正月十五 isPaid:False
Get track file url:http://www.ximalaya.com/tracks/8116248.json index:15
Request url:http://www.ximalaya.com/tracks/8116248.json
最新最火最好听流行歌曲 http://audio.xmcdn.com/group14/M0B/57/0C/wKgDZFXAE6OhR_